# PRMT-2270 14 vs 28 day cutoff with re-categorisation

In [1]:
import pandas as pd 
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
transfer_file_14_day_cutoff = "s3://prm-gp2gp-data-sandbox-dev/transfers-sample-6/2021-5-transfers_14_day_conversation_cutoff.parquet"
transfers_raw_14_day_cutoff = pd.read_parquet(transfer_file_14_day_cutoff)

transfers_14_day_cutoff = transfers_raw_14_day_cutoff.copy()
transfers_14_day_cutoff["status"] = transfers_14_day_cutoff["status"].str.replace("_", " ").str.title()

In [3]:
transfer_file_28_day_cutoff = "s3://prm-gp2gp-data-sandbox-dev/transfers-sample-6/2021-5-transfers_28_day_conversation_cutoff.parquet"
transfers_raw_28_day_cutoff = pd.read_parquet(transfer_file_28_day_cutoff)

transfers_28_day_cutoff = transfers_raw_28_day_cutoff.copy()
transfers_28_day_cutoff["status"] = transfers_28_day_cutoff["status"].str.replace("_", " ").str.title()

In [4]:
outcome_counts_14_day_cutoff = transfers_14_day_cutoff.fillna("N/A").groupby(by=["status", "failure_reason"]).agg({"conversation_id": "count"})
outcome_counts_14_day_cutoff = outcome_counts_14_day_cutoff.rename({"conversation_id": "Number of transfers", "failure_reason": "Failure Reason"}, axis=1)
outcome_counts_14_day_cutoff["% of transfers"] = (outcome_counts_14_day_cutoff["Number of transfers"] / outcome_counts_14_day_cutoff["Number of transfers"].sum()).multiply(100)

In [5]:
outcome_counts_14_day_cutoff

Number of transfers  \
status               failure_reason                                                 
Integrated On Time   N/A                                                   193033   
Process Failure      Integrated Late                                         6801   
                     Transferred, not integrated                            10312   
Technical Failure    COPC(s) not Acknowledged                                 199   
                     COPC(s) not sent                                          24   
                     Contains Fatal Sender Error                             3182   
                     Core Extract not Sent                                   3284   
                     Final Error                                             1904   
                     Request not Acknowledged                                 783   
Unclassified Failure Ambiguous COPC messages                                  305   
                     Transferred, not integrated, with error                  146   

                                                              % of transfers  
status               failure_reason                                           
Integrated On Time   N/A                                           87.753042  
Process Failure      Integrated Late                                3.091743  
                     Transferred, not integrated                    4.687848  
Technical Failure    COPC(s) not Acknowledged                       0.090466  
                     COPC(s) not sent                               0.010910  
                     Contains Fatal Sender Error                    1.446541  
                     Core Extract not Sent                          1.492910  
                     Final Error                                    0.865561  
                     Request not Acknowledged                       0.355953  
Unclassified Failure Ambiguous COPC messages                        0.138653  
                     Transferred, not integrated, with error        0.066372

In [6]:
outcome_counts_28_day_cutoff = transfers_28_day_cutoff.fillna("N/A").groupby(by=["status", "failure_reason"]).agg({"conversation_id": "count"})
outcome_counts_28_day_cutoff = outcome_counts_28_day_cutoff.rename({"conversation_id": "Number of transfers", "failure_reason": "Failure Reason"}, axis=1).astype('int32')
outcome_counts_28_day_cutoff["% of transfers"] = (outcome_counts_28_day_cutoff["Number of transfers"] / outcome_counts_28_day_cutoff["Number of transfers"].sum()).multiply(100)

In [7]:
outcome_counts_28_day_cutoff

Number of transfers  \
status               failure_reason                                                 
Integrated On Time   N/A                                                   193221   
Process Failure      Integrated Late                                        11235   
                     Transferred, not integrated                             5908   
Technical Failure    COPC(s) not Acknowledged                                 123   
                     COPC(s) not sent                                          19   
                     Contains Fatal Sender Error                             3182   
                     Core Extract not Sent                                   3248   
                     Final Error                                             1984   
                     Request not Acknowledged                                 741   
Unclassified Failure Ambiguous COPC messages                                  212   
                     Transferred, not integrated, with error                  100   

                                                              % of transfers  
status               failure_reason                                           
Integrated On Time   N/A                                           87.838507  
Process Failure      Integrated Late                                5.107445  
                     Transferred, not integrated                    2.685784  
Technical Failure    COPC(s) not Acknowledged                       0.055916  
                     COPC(s) not sent                               0.008637  
                     Contains Fatal Sender Error                    1.446541  
                     Core Extract not Sent                          1.476545  
                     Final Error                                    0.901929  
                     Request not Acknowledged                       0.336860  
Unclassified Failure Ambiguous COPC messages                        0.096375  
                     Transferred, not integrated, with error        0.045460

In [8]:
# High level summary of diff based on status
transfers_28_day_cutoff.fillna("N/A").groupby(by=["status", "failure_reason"]).agg({"conversation_id": "count"}).rename(columns={"conversation_id": "total difference"}) - transfers_14_day_cutoff.fillna("N/A").groupby(by=["status", "failure_reason"]).agg({"conversation_id": "count"}).rename(columns={"conversation_id": "total difference"})

total difference
status               failure_reason                                           
Integrated On Time   N/A                                                   188
Process Failure      Integrated Late                                      4434
                     Transferred, not integrated                         -4404
Technical Failure    COPC(s) not Acknowledged                              -76
                     COPC(s) not sent                                       -5
                     Contains Fatal Sender Error                             0
                     Core Extract not Sent                                 -36
                     Final Error                                            80
                     Request not Acknowledged                              -42
Unclassified Failure Ambiguous COPC messages                               -93
                     Transferred, not integrated, with error               -46

In [9]:
outcome = outcome_counts_14_day_cutoff.compare(outcome_counts_28_day_cutoff, keep_equal=True, keep_shape=True).round(2).rename(columns={"self":"14 day cutoff","other":"28 day cutoff"})
outcome["Difference"] = (outcome["Number of transfers"]["28 day cutoff"] - outcome["Number of transfers"]["14 day cutoff"]).astype('int32')
outcome["% Difference"] = (outcome["% of transfers"]["28 day cutoff"] - outcome["% of transfers"]["14 day cutoff"])

outcome[[
 ('Number of transfers', '14 day cutoff'),
 ('Number of transfers', '28 day cutoff'),
 ('Difference', ''),     
 ('% of transfers', '14 day cutoff'),
 ('% of transfers', '28 day cutoff'),
 ('% Difference', '')         
 ]]

Number of transfers  \
                                                                   14 day cutoff   
status               failure_reason                                                
Integrated On Time   N/A                                                  193033   
Process Failure      Integrated Late                                        6801   
                     Transferred, not integrated                           10312   
Technical Failure    COPC(s) not Acknowledged                                199   
                     COPC(s) not sent                                         24   
                     Contains Fatal Sender Error                            3182   
                     Core Extract not Sent                                  3284   
                     Final Error                                            1904   
                     Request not Acknowledged                                783   
Unclassified Failure Ambiguous COPC messages                                 305   
                     Transferred, not integrated, with error                 146   

                                                                            \
                                                             28 day cutoff   
status               failure_reason                                          
Integrated On Time   N/A                                            193221   
Process Failure      Integrated Late                                 11235   
                     Transferred, not integrated                      5908   
Technical Failure    COPC(s) not Acknowledged                          123   
                     COPC(s) not sent                                   19   
                     Contains Fatal Sender Error                      3182   
                     Core Extract not Sent                            3248   
                     Final Error                                      1984   
                     Request not Acknowledged                          741   
Unclassified Failure Ambiguous COPC messages                           212   
                     Transferred, not integrated, with error           100   

                                                             Difference  \
                                                                          
status               failure_reason                                       
Integrated On Time   N/A                                            188   
Process Failure      Integrated Late                               4434   
                     Transferred, not integrated                  -4404   
Technical Failure    COPC(s) not Acknowledged                       -76   
                     COPC(s) not sent                                -5   
                     Contains Fatal Sender Error                      0   
                     Core Extract not Sent                          -36   
                     Final Error                                     80   
                     Request not Acknowledged                       -42   
Unclassified Failure Ambiguous COPC messages                        -93   
                     Transferred, not integrated, with error        -46   

                                                             % of transfers  \
                                                              14 day cutoff   
status               failure_reason                                           
Integrated On Time   N/A                                              87.75   
Process Failure      Integrated Late                                   3.09   
                     Transferred, not integrated                       4.69   
Technical Failure    COPC(s) not Acknowledged                          0.09   
                     COPC(s) not sent                                  0.01   
                     Contains Fatal Sender Error                       1.45   
                     Core Ext